In [7]:
import numpy as np
import pandas as pd
import json

get dataframe

In [8]:
import numpy as np
import pandas as pd
import json
import requests
import time
import os
from SPARQLWrapper import SPARQLWrapper, JSON

def get_sparql_dataframe(service, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    sparql = SPARQLWrapper(service)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

q = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX : <http://h-da.de/fbi/artontology/>

SELECT ?artwork ?url ?name ?abstract (group_concat(?motif;separator=';') as ?motifs)
WHERE {
  ?artwork rdf:type :artwork;
     rdfs:label ?name;
     :image ?url;
     :abstract ?abstract;
     :motif/rdfs:label ?motif.
}
group by ?artwork ?url ?name ?abstract
"""

ds = "http://neuds.de:3030/artontology"
df = get_sparql_dataframe(ds, q)
df['id'] = df['artwork'].map(lambda x: x.split('/')[-1])
df.head()

,artwork,url,name,abstract,motifs,id
0,http://www.wikidata.org/entity/Q29019486,https://upload.wikimedia.org/wikipedia/commons...,Saint Proculus of Pozzuoli and his mother Sant...,Saints Proculus and Nicea is a 1636-1637 paint...,Nicea;Saint Procolo;Saint Procolo and Nicea,Q29019486
1,http://www.wikidata.org/entity/Q7527543,https://upload.wikimedia.org/wikipedia/commons...,Statue of John A. Macdonald,The Sir John A. Macdonald statue is a bronze s...,John A. Macdonald,Q7527543
2,http://www.wikidata.org/entity/Q3944494,https://upload.wikimedia.org/wikipedia/commons...,The Holy Family with Saint Catherine of Alexan...,Holy Family with Saint Catherine of Alexandria...,Virgin Mary;Catherine of Alexandria;boy;Child ...,Q3944494
3,http://www.wikidata.org/entity/Q368788,https://upload.wikimedia.org/wikipedia/commons...,Pietà,Pietà is a painting by the Italian Renaissance...,Virgin Mary;Jesus Christ;man;woman,Q368788
4,http://www.wikidata.org/entity/Q3630743,https://upload.wikimedia.org/wikipedia/commons...,Self-Portrait at the Age of 63,Self-Portrait at the Age of 63 is a self-portr...,Rembrandt;man,Q3630743


load bocv

In [83]:
from gensim.matutils import cossim as csim
from gensim.matutils import hellinger as hsim
from gensim.matutils import jaccard as jsim

path_bocv = 'art_bocv.json'
with open(path_bocv) as f:
  bocv_dict = json.load(f)

def bow2vec(bocv):
  vec = np.zeros(100)
  for k in bocv.keys():
    vec[int(k)] = int(bocv[k])
  return(vec)

def norm_bow(bow):
  out = bow.copy()
  size = 0
  for k in bow.keys():
    size += bow[k]
  for k in bow.keys():
    out[k] = out[k] / size
  return(out)

def csim(bocv1, bocv2):
  vec1 = bow2vec(bocv1)
  vec2 = bow2vec(bocv2)
  l1 = np.linalg.norm(vec1)
  l2 = np.linalg.norm(vec2)
  score = np.dot(vec1, vec2) / (l1 * l2)
  return(score)

def score_bocv(bocv1, bocv2):
  score = csim(bocv1, bocv2)
  score = np.around(score, 5)
  return(score)

def score_bocv(bocv1, bocv2):
  vec1 = bow2vec(bocv1)
  vec2 = bow2vec(bocv2)
  score = jsim(vec1, vec2)
  score = np.around(score, 5)
  return(score)

In [88]:
id1 = 'Q1231009'
id2 = 'Q24976702'
bocv1 = bocv_dict[id1]
bocv2 = bocv_dict[id2]
bocv1 = {'1': 1}
bocv2 = {'1': 2, '2': 1}
score = score_bocv(bocv1, bocv2)
print(bocv1)
print(bocv2)
print(score)

{'1': 1}
{'1': 2, '2': 1}
0.33333


load lda

In [92]:
import pickle
from gensim.matutils import cossim as csim

lda_model = pickle.load(open('nlp_lda.pkl', 'rb'))
lda_dict = pickle.load(open('nlp_dict.pkl', 'rb'))

def score_lda(s1, s2, lda_model, lda_dict):
  '''
  input: abstract values from artworks df
  '''
  v1 = s1.lower().split()
  v2 = s2.lower().split()
  bow1 = lda_dict.doc2bow(v1)
  bow2 = lda_dict.doc2bow(v2)
  lda1 = lda_model[bow1]
  lda2 = lda_model[bow2]
  score = csim(lda1, lda2)
  score = np.around(score, 5)
  return(score)

create list of tuples

In [90]:
aids = df['id'].values
tuples = []
n = len(aids)
for i, aid1 in enumerate(aids):
  for j in range(i + 1, n):
    tuples.append([aid1, aids[j]])

add scores to elasticsearch

In [93]:
from IPython.display import clear_output
from elasticsearch import Elasticsearch, helpers
es = Elasticsearch([{'host':'neuds.de', 'port': 15660}])

bulk = []
counter = 0
for i, t in enumerate(tuples):
  id1 = t[0]
  id2 = t[1]
  # score lda
  s1 = df[df['id'] == id1]['abstract'].values[0]
  s2 = df[df['id'] == id2]['abstract'].values[0]
  score_nlp = score_lda(s1, s2, lda_model, lda_dict)
  # score cv
  bocv1 = bocv_dict[id1]
  bocv2 = bocv_dict[id2]
  score_cv = score_bocv(bocv1, bocv2)
  # old
  # # combine score
  # if score_cv > 0.3:
  #   score = (score_cv + score_nlp) / 2
  # else:
  #   score = score_nlp
  score = (score_cv + score_nlp) / 2
  # gen result
  result = {'_id': i, 'id1': id1, 'id2': id2, 'score': score}
  # load result to bulk
  bulk.append(result)
  # upload with bulk size 5000
  counter += 1
  if (counter % 5000) == 0:
    helpers.bulk(es, bulk, index='artsimj')
    clear_output()
    bulk = []
    print(f'ES upload at: {counter}')
# final save
helpers.bulk(es, bulk, index='artsimj')
clear_output()
print(f'ES upload at: {counter}')

ES upload at: 10976955


## tests

In [43]:
id1 = 'Q179900'
id2 = 'Q205259'
bocv1 = bocv_dict[id1]
bocv2 = bocv_dict[id2]
score = score_bocv(bocv1, bocv2)
print(bocv1)
print(bocv2)
print(score)

{'1': 1, '75': 1}
{'1': 1, '86': 1}
0.5


In [42]:
bocv1 = {'1': 1, '76': 1}
bocv2 = {'1': 5, '75': 1}
score = score_bocv(bocv1, bocv2)
print(score)

0.69338


In [ ]:
import pickle
lda = pickle.load(open('lda_text.pkl', 'rb'))

s1 = df[df['id'] == 'Q179900']['abstract'].values[0].lower().split()
s2 = df[df['id'] == 'Q205259']['abstract'].values[0].lower().split()
v1 = art_dict.doc2bow(s1)
v2 = art_dict.doc2bow(s2)
lda1 = lda[v1]
lda2 = lda[v2]
score = csim(lda1, lda2)
print(score)

In [51]:
from elasticsearch import Elasticsearch, helpers
es = Elasticsearch([{'host':'neuds.de', 'port': 15660}])

doc = [
  {'_id': 1, 'score': 0.74, 'id1': 'Q123', 'id2': 'Q332'},
  {'_id': 2, 'score': 0.57, 'id1': 'Q124', 'id2': 'Q322'},
  {'_id': 3, 'score': 0.41, 'id1': 'Q125', 'id2': 'Q312'},
  {'_id': 4, 'score': 0.98, 'id1': 'Q126', 'id2': 'Q302'},
]

helpers.bulk(es, doc, index='art-test3')

(4, [])

## test of similarity
napoleon
`Q1231009`
hermann
`Q24976702`

In [14]:
s1 = df[df['id'] == 'Q1231009']['abstract'].values[0].lower().split()
s2 = df[df['id'] == 'Q24976702']['abstract'].values[0].lower().split()
v1 = lda_dict.doc2bow(s1)
v2 = lda_dict.doc2bow(s2)
lda1 = lda_model[v1]
lda2 = lda_model[v2]
score = csim(lda1, lda2)
print(score)

0.9786006239487305


In [44]:
id1 = 'Q1231009'
id2 = 'Q24976702'
bocv1 = bocv_dict[id1]
bocv2 = bocv_dict[id2]
score = score_bocv(bocv1, bocv2)
print(bocv1)
print(bocv2)
print(score)

{'1': 13, '85': 1, '19': 1}
{'1': 1}
0.99413
